In [ ]:
import streamlit as st
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
def show():
    st.title("Data Insights")

    conn = sqlite3.connect('Campus_Placements.db')

    queries = {
    "1. Avg Programming Problems solved per Batch": """
        SELECT S.Course_Batch, AVG(P.Problems_solved) as Avg_Score
        FROM STUDENTS_TABLE S
        JOIN Programming_Table P ON S.Student_ID = P.Student_ID
        GROUP BY S.Course_Batch;
    """,

    "2. Top 5 Placement-Ready Students": """
        SELECT S.Name, P.Problems_solved,(SS.Communication+ SS.Team_Work + SS.Presentation+
        SS.Leadership+ SS.Critical_Thinking + SS.Interpersonal_Skills) AS Soft_skills_Score
        FROM STUDENTS_TABLE S
        JOIN Programming_Table P ON S.Student_ID = P.Student_ID
        JOIN Soft_Skills_Table SS ON S.Student_ID = SS.Student_ID
        ORDER BY P.Problems_solved DESC, Soft_skills_Score DESC
        LIMIT 5;
    """,

    "3. Distribution of Soft Skills Scores": """
        SELECT (SS.Communication+ SS.Team_Work + SS.Presentation+
        SS.Leadership+ SS.Critical_Thinking + SS.Interpersonal_Skills) AS Soft_skills_Score FROM Soft_Skills_Table SS;
    """,

    "4. Average CGPA by Graduation Year": """
        SELECT AVG(S.CGPA) AS Avg_CGPA, S.Graduation_Year
        FROM STUDENTS_TABLE S
        GROUP BY S.Graduation_Year
        ORDER BY Avg_CGPA DESC;
    """,

    "5. Percentage of Students Placed vs Unplaced": """
        SELECT Placement_Status, COUNT(*) AS Total_Students,
        ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM Placements_Table), 2) AS Percentage
        FROM Placements_Table
        GROUP BY Placement_Status;
    """,

    "6. Top 3 Companies Offering Highest Placement Packages": """
        SELECT Company_Name, Placement_Package_INR_LPA
        FROM Placements_Table 
        WHERE Company_Name != 'Not Applicable'
        ORDER BY Placement_Package_INR_LPA DESC
        LIMIT 3;
    """,

    "7. Average Mock Interview Score by Placement Status": """
        SELECT Placement_Status, AVG(Mock_Interview_Score) AS Avg_Mock_Score
        FROM Placements_Table
        GROUP BY Placement_Status;
    """,

    "8. Top 5 Students with High CGPA and Excellent Attandance": """
        SELECT Name, Age, CGPA, Attandance 
        FROM STUDENTS_TABLE
        WHERE Attandance > 85
        ORDER BY CGPA DESC
        LIMIT 5;
    """,

    "9. Programming Language vs Placement Status": """
        SELECT P.Language, PL.Placement_Status, COUNT(*) AS Total_Students
        FROM Programming_Table P
        JOIN Placements_Table PL ON P.Student_ID = PL.Student_ID
         GROUP BY P.Language, PL.Placement_Status;

    """,

    "10. Internship Impact on Placement Package": """
        SELECT 
        CASE WHEN Internships_Completed >= 1 THEN 'With Internship' ELSE 'Without Internship' END AS Internship_Status,
        AVG(Placement_Package_INR_LPA) AS Avg_Placement_Package
        FROM Placements_Table
        WHERE Placement_Package_INR_LPA != 'Not Applicable'
        GROUP BY Internship_Status;
    """
}


    selected_query = st.selectbox("Select an Insight:", list(queries.keys()))
    df = pd.read_sql(queries[selected_query], conn)
    st.dataframe(df)

    # Visualization for data insights
    if selected_query == "1. Avg Programming Problems solved per Batch":
        plt.bar(df['Course_Batch'], df['Avg_Score'], color='orange')
        plt.title("Average Problems Solved per Batch")
        plt.xlabel("Course Batch")
        plt.ylabel("Avg Problems Solved")
        st.pyplot(plt)

    elif selected_query == "2. Top 5 Placement-Ready Students":
        plt.bar(df['Name'], df['Problems_solved'], color='green', label='Problems Solved')
        plt.xlabel("Problems Solved")
        plt.title("Top 5 Placement-Ready Students (by Problems Solved)")
        st.pyplot(plt)

        
    elif selected_query == "3. Distribution of Soft Skills Scores":
        plt.hist(df['Soft_skills_Score'], bins=10, color='skyblue')
        plt.title("Soft Skills Score Distribution")
        st.pyplot(plt)
        
    elif selected_query=="4. Average CGPA by Graduation Year":
        plt.bar(df['Graduation_Year'], df['Avg_CGPA'], color='purple')
        plt.title("Average CGPA by Graduation Year")
        plt.xlabel("Graduation Year")
        plt.ylabel("Average CGPA")
        st.pyplot(plt)
    
    elif selected_query=="5. Percentage of Students Placed vs Unplaced":
        plt.pie(df['Percentage'], labels=df['Placement_Status'], autopct='%1.1f%%', colors=['lightgreen', 'lightcoral'])
        plt.title("Placement Status Distribution")
        st.pyplot(plt)
    
    elif selected_query=="6. Top 3 Companies Offering Highest Placement Packages":
        plt.barh(df['Company_Name'], df['Placement_Package_INR_LPA'], color='gold')
        plt.title("Top 3 Companies by Placement Package")
        plt.xlabel("Package (INR LPA)")
        st.pyplot(plt)
        
    elif selected_query=="7. Average Mock Interview Score by Placement Status":
        plt.bar(df['Placement_Status'], df['Avg_Mock_Score'], color='cyan')
        plt.title("Average Mock Interview Score by Placement Status")
        plt.xlabel("Placement Status")
        plt.ylabel("Average Mock Score")
        st.pyplot(plt)
    
    elif selected_query=="8. Top 5 Students with High CGPA and Excellent Attendance":
        plt.scatter(df['Attendance'], df['CGPA'], color='magenta')
        plt.title("CGPA vs Attendance (Top 5 Students)")
        plt.xlabel("Attendance (%)")
        plt.ylabel("CGPA")
        for i in range(len(df)):
            plt.text(df['Attendance'][i]+0.1, df['CGPA'][i], df['Name'][i], fontsize=8)
        st.pyplot(plt)
    
    elif selected_query=="9. Programming Language vs Placement Status":
        import seaborn as sns
        plt.figure(figsize=(8,5))
        sns.barplot(x='Language', y='Total_Students', hue='Placement_Status', data=df)
        plt.title("Programming Language vs Placement Status")
        plt.xlabel("Programming Language")
        plt.ylabel("Number of Students")
        st.pyplot(plt)

        
    elif selected_query=="10. Internship Impact on Placement Package":
        plt.bar(df['Internship_Status'], df['Avg_Placement_Package'], color='teal')
        plt.title("Internship Impact on Placement Package")
        plt.xlabel("Internship Status")
        plt.ylabel("Avg Package (INR LPA)")
        st.pyplot(plt)
    
    
    conn.close()
